## Single-cell RNA-seq Analysis Workflow

This notebook presents a pipeline for analyzing single-cell RNA sequencing (scRNA-seq) data using the `Scanpy` framework. Below are the essential Python packages that will be used throughout the analysis.

### Import Required Libraries

In [ ]:
import numpy as np   
import pandas as pd
import anndata as ad 
import scanpy as sc
import seaborn as sb
import matplotlib.pyplot as plt
from scipy import sparse

### Load Raw Expression Matrix

In [ ]:
adata = sc.read_h5('raw_matrix.h5')

### Quality Control (QC) Metrics Calculation and Visualization

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('mt-') 
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=2, multi_panel=True)
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')

sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')


In [ ]:
sc.pp.filter_cells(adata, min_genes=300) 
sc.pp.filter_genes(adata, min_cells=3) 
adata = adata[adata.obs.n_genes_by_counts < 4000, :]
adata = adata[adata.obs.total_counts < 30000, :]
adata = adata[adata.obs.pct_counts_mt < 10, :]

### Doublet Detection Using Scrublet

In [ ]:
import scrublet as scr
scrub = scr.Scrublet(adata.X)
doublet_scores, predicted_doublets = scrub.scrub_doublets()
adata.obs['doublet_scores'] = doublet_scores
adata.obs['predicted_doublets'] = predicted_doublets
scrub.plot_histogram()
plt.title('Doublet Score Distribution')
plt.show()

In [ ]:
threshold = 0.57  
adata.obs['predicted_doublets'] = doublet_scores > threshold
print(adata.obs['predicted_doublets'].value_counts())
adata = adata[adata.obs['predicted_doublets'] == False, :]

#### Preserve Raw Counts in `adata.layers`

In [ ]:
adata.layers["Raw_counts"] = adata.X.copy()

#### Normalization and Log Transformation

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4) 
sc.pp.log1p(adata) 

#### Identification of Highly Variable Genes and Data Scaling

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5,batch_key='batch',n_top_genes=2000)
sc.pl.highly_variable_genes(adata)
adata.raw = adata
adata = adata[:, adata.var.highly_variable] 
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])  
sc.pp.scale(adata, max_value=10) 

#### Principal Component Analysis (PCA)

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca(adata, color='Ptprc')
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50)

In [ ]:
import harmonypy as hm
dfd = ['batch']
harmony_out = hm.run_harmony(adata.obsm['X_pca'], adata.obs,dfd, max_iter_harmony = 50)  
adata.obsm['X_pca_harmony'] = harmony_out.Z_corr.T

#### Clustering and Visualization with UMAP

After dimensionality reduction, we perform clustering and visualize the cell populations using UMAP.


In [ ]:
sc.pp.neighbors(adata,use_rep='X_pca_harmony', n_neighbors=15, n_pcs=40) 
sc.tl.umap(adata)
sc.pl.umap(adata,color=['class'])
sc.tl.leiden(adata,resolution=1) 
sc.pl.umap(adata, color=['leiden'])
sc.pl.umap(adata, color=['leiden'],legend_loc='on data')

In [58]:
marker_genes_dict = {
 'Immnue': ['Ptprc','Itgam','Cd3e','Gata3','Trdc'],
 'Fibroblast': ['Vim','Col1a1','Acta2','Pdpn'],
 'Endo': ['Pecam1','Cdh5'],
 'Epi': ['Epcam'],
  'Glia': ['Sox10'],
 }

In [ ]:
ax = sc.pl.dotplot(adata, marker_genes_dict, groupby='leiden',dendrogram=True, swap_axes=False, 
                   standard_scale="var",cmap='PiYG_r', 
                      dot_min=0,  
                     dot_max=1,  
                     vmin=0,  
                      vmax=1 )  

#### Remove Unwanted Clusters and Re-cluster

In [ ]:
clusters_to_remove = ['7', '8', '17', '22', '27', '28', '29', '30', '31']
adata = adata[~adata.obs['leiden'].isin(clusters_to_remove)].copy()
sc.tl.leiden(adata,resolution=1) 
sc.pl.umap(adata, color=['leiden'])
sc.pl.umap(adata, color=['leiden'],legend_loc='on data')
sc.tl.dendrogram(adata,groupby='leiden')
ax = sc.pl.dotplot(adata, marker_genes_dict, groupby='leiden',dendrogram=True, swap_axes=False, 
                   standard_scale="var",cmap='PiYG_r', 
                      dot_min=0,  
                     dot_max=1,   
                     vmin=0,  
                      vmax=1 )   

In [ ]:
clusters_to_remove = ['3', '10', '11', '14', '17', '19']
adata = adata[~adata.obs['leiden'].isin(clusters_to_remove)].copy()

In [ ]:
adata = adata.raw.to_adata()  

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5,n_top_genes=2500)
adata.raw = adata  
adata = adata[:, adata.var.highly_variable] 

####  Batch Effect Correction with Harmony

In [ ]:
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt']) 
sc.pp.scale(adata, max_value=10) 
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca(adata, color='Ptprc')
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50)
import harmonypy as hm
dfd = ['batch']
harmony_out = hm.run_harmony(adata.obsm['X_pca'], adata.obs,dfd, max_iter_harmony = 50) 
adata.obsm['X_pca_harmony'] = harmony_out.Z_corr.T

#### Dimensionality Reduction, Diffusion Map, and Clustering

In [ ]:
sc.pp.neighbors(adata,use_rep='X_pca_harmony', n_neighbors=15, n_pcs=25)
sc.tl.umap(adata, min_dist=0.4, spread=2,random_state=57)
sc.pl.umap(adata, color='S100a9', return_fig=True, show=False)
sc.tl.diffmap(adata)
sc.pl.diffmap(adata, color='S100a9')
adata = adata.raw.to_adata()  
adata.obs['n_expressed_genes'] = (adata.X > 0).sum(axis=1).A1
sc.tl.leiden(adata,resolution=0.3) 
sc.pl.umap(adata, color=['leiden'])
sc.pl.umap(adata, color=['leiden'],legend_loc='on data', legend_fontsize=10)